# Manual mobilenet implementation

In [1]:
from models.mobilenet import ZkMobileNet
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pytorch_lightning as pl
import torchvision
import torchvision.transforms.v2 as transforms
import json
import os

In [2]:
# Circom defines the range of positives are [0, p/2] and the range of negatives are [(p/2)+1, (p-1)].
# CIRCOM_PRIME = 21888242871839275222246405745257275088548364400416034343698204186575808495617
CIRCOM_PRIME = 28948022309329048855892746252171976963363056481941647379679742748393362948097
MAX_POSITIVE = CIRCOM_PRIME // 2
MAX_NEGATIVE = MAX_POSITIVE + 1 # The most positive number
CIRCOM_NEGATIVE_1 = CIRCOM_PRIME - 1
EXPONENT = 15

def from_circom(x):
    if type(x) != int:
        x = int(x)
    if x > MAX_POSITIVE: 
        return x - CIRCOM_PRIME
    return x
    
def to_circom(x):
    return x % CIRCOM_PRIME
    
def to_circom_input(array: np.array): 
    if type(array) != np.array:
        array = np.array(array)
    int_array = array.round().astype(int)
    int_array = to_circom(int_array)
    return int_array.astype(str).tolist()


def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    # out = np.zeros((outRows, outCols, nFilters))
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    # remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += int(input[row*strides+x, col*strides+y, channel]) * int(weights[x, y, channel])
                
                out[row][col][channel] += int(bias[channel])
                remainder[row][col][channel] = str(int(out[row][col][channel] % n))
                out[row][col][channel] = int(out[row][col][channel] // n)
                            
    return out, remainder

def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    str_out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row][col][filter] += int(input[row*strides, col*strides, k]) * int(weights[k, filter])
                    
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = int(out[row][col][filter] // n)
                str_out[row][col][filter] = str(out[row][col][filter] % p)
                            
    return out, str_out, remainder

def SeparableConvImpl(nRows, nCols, nChannels, nDepthFilters, nPointFilters, kernelSize, strides, n, input, depthWeights, pointWeights, depthBias, pointBias):
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1

    depth_out, depth_remainder = DepthwiseConv(nRows, nCols, nChannels, nDepthFilters, kernelSize, strides, n, input, depthWeights, depthBias)
    point_out, point_str_out, point_remainder = PointwiseConv2d(outRows, outCols, nChannels, nPointFilters, strides, n, depth_out, pointWeights, pointBias)
    return depth_out, depth_remainder, point_out, point_str_out, point_remainder

In [3]:
class DatasetWrapper(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

testset = torchvision.datasets.CIFAR10(
    root='./CIFAR10', train=False, download=True, transform=transform)

trainset = torchvision.datasets.CIFAR10(
    root='./CIFAR10', train=True, download=True)

# split the train set into train/validation
train_set_size = int(len(trainset) * 0.8)
valid_set_size = len(trainset) - train_set_size

seed = torch.Generator().manual_seed(42)
trainset, validset = torch.utils.data.random_split(trainset, [train_set_size, valid_set_size], generator=seed)

trainset = DatasetWrapper(trainset, transform)
validset = DatasetWrapper(validset, transform)

# Create train dataloader
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=512, shuffle=True, num_workers=24)

# Create validation dataloader
validloader = torch.utils.data.DataLoader(
    validset, batch_size=512, shuffle=False, num_workers=24)

# Create test dataloader
testloader = torch.utils.data.DataLoader(
    testset, batch_size=512, shuffle=False, num_workers=24)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

/home/semar/.local/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [14]:
MODEL_WEIGHTS_PATH = "./checkpoints/no_padding_100epochs.pth", 

model = ZkMobileNet(trainloader, num_classes=10, alpha=0.25, max_epochs=100)

saved = torch.load("./checkpoints/no_padding_100epochs.pth")
# model.load_state_dict(saved['state_dict'])
model.load_state_dict(saved['net'])
model.eval()

image, label = validset[0]
image = image.unsqueeze(0)
logits = model(image)
pred_idx = logits.argmax()

print(f"Predicted {classes[pred_idx]} - idx: {pred_idx}")

Predicted horse - idx: 7


In [15]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    p = CIRCOM_PRIME
    Input = [[[str(int(input[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(int(weights[i][j][k][l]) % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(int(bias[i]) % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += int(input[i*strides+x][j*strides+y][k]) * int(weights[x][y][k][m])
                out[i][j][m] += int(bias[m])
                remainder[i][j][m] = str(int(out[i][j][m]) % n)
                out[i][j][m] = str(int(out[i][j][m]) // n % p)
    return Input, Weights, Bias, out, remainder

In [16]:
def BatchNormalizationInt(nRows, nCols, nChannels, n, X_in, a_in, b_in):
    p = CIRCOM_PRIME
    X = [[[str(int(X_in[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    A = [str(int(a_in[k]) % p) for k in range(nChannels)]
    B = [str(int(b_in[k]) % p) for k in range(nChannels)]
    out = [[[0 for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
    remainder = [[[None for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
    for i in range(nRows):
        for j in range(nCols):
            for k in range(nChannels):
                out[i][j][k] = int(int(X_in[i][j][k]) * int(a_in[k]) + int(b_in[k]))
                remainder[i][j][k] = str(int(out[i][j][k]) % n)
                out[i][j][k] = int(out[i][j][k] // n)
    return X, A, B, out, remainder

In [17]:
gamma = model.bn.weight
beta = model.bn.bias
mean = model.bn.running_mean
var = model.bn.running_var
eps = model.bn.eps

a = (gamma/(var+eps)**.5).detach()
b = (beta-gamma*mean/(var+eps)**.5).detach().tolist()

image, label = testset[0]
image = image.unsqueeze(0)
out = model.conv(image)
expected = model.bn(out)
# out = out.squeeze()
print(f"{out.shape=}")

expected = torch.permute(expected.squeeze(), (1, 2, 0))

print(f"{expected.shape=}")

quantized_in = torch.permute(out.squeeze(), (1, 2, 0)) * 10**EXPONENT
quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]
print(out.shape)

X, A, B, actual, remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in actual]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

out.shape=torch.Size([1, 8, 32, 32])
expected.shape=torch.Size([32, 32, 8])
torch.Size([1, 8, 32, 32])


# Testing head layer

In [18]:
# CONVOLUTION LAYER
weights = model.conv.weight.detach().numpy().transpose(2, 3, 1, 0)
image, label = testset[0]

expected = model.conv(image).detach().numpy()
bias = torch.zeros(weights.shape[3]).numpy()

# padded = pad(image, 1).transpose(1,2,0)
padded = F.pad(image, (1,1,1,1), "constant", 0).numpy()
padded = padded.transpose(1,2,0)

quantized_image = (padded * 10**EXPONENT).round()
quantized_weights = (weights * 10**EXPONENT).round() # .transpose(0, 3, 1, 0) # [nFilters, nChannels, H, W] -> 

circuit_in, circuit_conv_weights, circuit_conv_bias, circuit_conv_out, circuit_conv_remainder = Conv2DInt(34, 34, 3, 8, 3, 1, 10**EXPONENT, quantized_image, quantized_weights, bias)
# output, remainder = Conv2DInt(34, 34, 3, 8, 3, 1, 10**EXPONENT, quantized_image, quantized_weights, bias)
# test_output = output / 10**(EXPONENT)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_conv_out]

expected = expected.transpose((1, 2, 0))

assert(np.allclose(test_output, expected, atol=1e-6))

# BATCH NORM CONSTANTS
gamma = model.bn.weight
beta = model.bn.bias
mean = model.bn.running_mean
var = model.bn.running_var
eps = model.bn.eps

a = (gamma/(var+eps)**.5).detach()
b = (beta-gamma*mean/(var+eps)**.5).detach().tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

# BATCH NORM USING PYTORCH OUTPUT
image, label = testset[0]
image = image.unsqueeze(0)
out = model.conv(image)
expected = model.bn(out)

expected = torch.permute(expected.squeeze(), (1, 2, 0))

quantized_in = torch.permute(out.squeeze(), (1, 2, 0)) * 10**EXPONENT

X, A, B, actual, remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]
assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# BATCH NORM USING CIRCUIT CONV OUTPUT
quantized_in = [[[from_circom(int(out)) for out in vec] for vec in matrix] for matrix in circuit_conv_out]

_, circuit_bn_a, circuit_bn_b, circuit_bn_out, circuit_bn_remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, quantized_in, quantized_a, quantized_b)

test_output = [[[int(from_circom(int(out))) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# RELU USING CIRCUIT OUTPUT
relu_in = [[[to_circom(bn_out) if bn_out < MAX_POSITIVE else 0 for bn_out in vec] for vec in matrix] for matrix in circuit_bn_out]
relu_out = [[[str(bn_out) if bn_out < MAX_POSITIVE else 0 for bn_out in vec] for vec in matrix] for matrix in relu_in]

input_json_path = "head_input1.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": circuit_in,
               "conv2d_weights": circuit_conv_weights,
               "conv2d_bias": circuit_conv_bias,
               "conv2d_out": circuit_conv_out,
               "conv2d_remainder": circuit_conv_remainder,
               
               "bn_a": circuit_bn_a,
               "bn_b": circuit_bn_b,
               "bn_out": circuit_bn_out,
               "bn_remainder": circuit_bn_remainder,
               
               "relu_out": relu_out,
               }, input_file)

os.chdir("circuits")
!./head/head_cpp/head ../head_input1.json head.wtns
# !npx snarkjs groth16 prove head/circuit_final.zkey head.wtns proof.json public_test.json
os.chdir("../")

print("ok")

something
before conv2d
after conv2d
after bn
at:  0 1 6
in:  1745789835369983
after relu
end
ok


# Testing padding over highly padded input (to try and fold the circuit using nova)

In [23]:
class SeparableConv2d(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2d, self).__init__()
        self.dw_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=0, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            # nn.ReLU(inplace=False),
        )
        self.pw_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            # nn.ReLU(inplace=False),
        )
        
input = torch.randn((1, 3, 5, 5))
test_model = SeparableConv2d(3, 6)

In [24]:
weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

expected = test_model.dw_conv[0](input).detach().numpy()
print(expected.shape)

padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
test_output = test_output[1:4, 1:4, :]
expected = expected.squeeze().transpose((1, 2, 0))
# expected = expected * 10**EXPONENT

print("test_output shape: ", test_output.shape)
print("expected shape: ", expected.shape)
assert(np.allclose(expected, test_output, atol=0.00001))

(1, 3, 3, 3)
test_output shape:  (3, 3, 3)
expected shape:  (3, 3, 3)


In [188]:
# input = torch.randn((1, 3, 5, 5))
# model = SeparableConv2d(3, 6)

In [25]:
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    # out = np.zeros((outRows, outCols, nFilters))
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    # remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += int(input[row*strides+x, col*strides+y, channel]) * int(weights[x, y, channel])
                
                out[row][col][channel] += int(bias[channel])
                remainder[row][col][channel] = str(int(out[row][col][channel] % n))
                out[row][col][channel] = int(out[row][col][channel] // n)
                            
    return out, remainder
    
weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

expected = test_model.dw_conv[0](input).detach().numpy()
print(expected.shape)

padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

# test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
# test_output = test_output[1:6, 1:6, :]
test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
test_output = test_output[1:4, 1:4, :]

expected = expected.squeeze().transpose((1, 2, 0))
# expected = expected * 10**EXPONENT
assert(np.allclose(expected, test_output, atol=0.00001))

circuit_in = quantized_image.round().astype(int).astype(str).tolist()
# circuit_weights = quantized_weights.round().astype(int).astype(str).tolist()
circuit_bias = bias.round().astype(int).astype(str).tolist()

p = CIRCOM_PRIME
Input = [[[str(int(quantized_image[i][j][k].round()) % p) for k in range(quantized_image.shape[2])] for j in range(quantized_image.shape[1])] for i in range(quantized_image.shape[0])]
Weights = [[[str(int(quantized_weights[i][j][k].round()) % p) for k in range(quantized_weights.shape[2])] for j in range(quantized_weights.shape[1])] for i in range(quantized_weights.shape[0])]
# Out = [[[str(int(actual[i][j][k]) % p) for k in range(3)] for j in range(7)] for i in range(7)]

test_output = np.array([[[int(out) for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
# test_output = test_output[1:6, 1:6, :]
Out = [[[str(int(test_output[i][j][k]) % p) for k in range(3)] for j in range(5)] for i in range(5)]
Rem = [[[str(rem[i][j][k]) for k in range(3)] for j in range(5)] for i in range(5)]

input_json_path = "backbone_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({
               "in": Input,
               "weights": Weights,
               "bias": circuit_bias,
               "remainder": Rem,
               "out": Out,
              },
              input_file)


# os.chdir("circuits")
# !./origDepthwiseConv2d/origDepthwiseConv2d_cpp/origDepthwiseConv2d ../backbone_input.json head.wtns
# !npx snarkjs groth16 prove ./origDepthwiseConv2d/circuit_final.zkey head.wtns proof.json public_test.json
# os.chdir("../")

print("OK")

(1, 3, 3, 3)
OK


# Padded Convolution test

# 

In [28]:
input = torch.randn((1, 3, 7, 7))
# model = SeparableConv2d(3, 6)
# model.eval()

In [26]:
def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row][col][filter] += int(input[row*strides][col*strides][k]) * int(weights[k, filter])
                    
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = int(out[row][col][filter] // n)
                            
    return out, remainder

In [34]:
# def PaddedDepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
#     assert(nFilters % nChannels == 0)
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     # out = np.zeros((outRows, outCols, nFilters))
#     Input = [[[str(int(input[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
#     Weights = [[[str(int(weights[i][j][k].round()) % p) for k in range(weights.shape[2])] for j in range(weights.shape[1])] for i in range(weights.shape[0])]
#     out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
#     Bias = bias.round().astype(int).astype(str).tolist()
#     out_str = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
#     remainder = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for channel in range(nChannels):
#                 for x in range(kernelSize):
#                     for y in range(kernelSize):
#                         out[row+1][col+1][channel] += int(input[row*strides+x][col*strides+y][channel]) * int(weights[x, y, channel])
                
#                 out[row+1][col+1][channel] += int(bias[channel])
#                 remainder[row+1][col+1][channel] = str(int(out[row+1][col+1][channel] % n))
#                 out[row+1][col+1][channel] = int(out[row+1][col+1][channel] // n)
#                 out_str[row+1][col+1][channel] = str(out[row+1][col+1][channel] % p)
                            
#     return out, remainder
    
weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

print("Input shape: ", input.shape)
expected = test_model.dw_conv(input).detach()
print("Expected shape: ", expected.shape)

weights = weights.transpose((1, 2, 0))

quantized_image = input.squeeze().numpy().transpose((1,2,0)) * 10**EXPONENT
# quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

# actual, rem = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)
circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)

test_output = np.array([[[int(value) / 10**EXPONENT for value in vec] for vec in matrix] for matrix in depth_out])
test_output = test_output[1:6, 1:6, :]

# expected = expected.squeeze().transpose((1, 2, 0))
# expected = F.pad(expected.squeeze(), (1,1,1,1)).numpy().transpose((1,2,0))
expected = expected.squeeze().numpy().transpose((1,2,0))
print("Expected shape: ", expected.shape)
print("test_output shape: ", test_output.shape)

# expected = expected * 10**EXPONENT
assert(np.allclose(expected, test_output, atol=0.00001))
print("--------------------------------")

print("Input shape: ", input.shape)
expected = test_model.dw_conv(input).detach()
print("expected shape: ", expected.shape)
expected = test_model.dw_conv(expected).detach()
expected = expected.squeeze().numpy().transpose((1,2,0))
print("expected shape: ", expected.shape)
dw_input, rem = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)
print("1st shape: ", np.array(dw_input).shape)
actual, rem = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, dw_input, quantized_weights.round(), bias)
print("2nd shape: ", np.array(actual).shape)
test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
test_output = test_output[2:5, 2:5, :]


# expected = F.pad(expected.squeeze(), (2,2,2,2)).numpy().transpose((1,2,0))
print("actual shape: ", np.array(actual).shape)
print("expected shape: ", np.array(expected).shape)
assert(np.allclose(expected, test_output, atol=0.00001))

p = CIRCOM_PRIME
# quantized_image = np.array(dw_input)
# Input = [[[str(int(dw_input[i][j][k]) % p) for k in range(quantized_image.shape[2])] for j in range(quantized_image.shape[1])] for i in range(quantized_image.shape[0])]
# Weights = [[[str(int(quantized_weights[i][j][k].round()) % p) for k in range(quantized_weights.shape[2])] for j in range(quantized_weights.shape[1])] for i in range(quantized_weights.shape[0])]
# # Out = [[[str(int(actual[i][j][k]) % p) for k in range(3)] for j in range(7)] for i in range(7)]

# test_output = np.array([[[int(out) for out in asdf] for asdf in asdfasdf] for asdfasdf in actual])
# print("test_output shape: ", np.array(test_output).shape)
# # # test_output = test_output[1:6, 1:6, :]
# Out = [[[str(int(test_output[i][j][k]) % p) for k in range(test_output.shape[2])] for j in range(test_output.shape[1])] for i in range(test_output.shape[0])]
# Rem = [[[str(rem[i][j][k]) for k in range(test_output.shape[2])] for j in range(test_output.shape[1])] for i in range(test_output.shape[0])]
# print("OUT SHAPE: ", np.array(Out).shape)
# print("REM SHAPE: ", np.array(Rem).shape)

# input_json_path = "padded_input.json"
# with open(input_json_path, "w") as input_file:
#     json.dump({
#                "in": Input,
#                "dw_conv_weights": Weights,
#                "dw_conv_bias": circuit_bias,
#                "dw_conv_remainder": Rem,
#                "dw_conv_out": Out,
#               },
#               input_file)


# os.chdir("circuits")
# !./padded/padded_cpp/padded ../padded_input.json head.wtns
# # # !npx snarkjs groth16 prove ./origDepthwiseConv2d/circuit_final.zkey head.wtns proof.json public_test.json
# os.chdir("../")

# # print("OK")

Input shape:  torch.Size([1, 3, 7, 7])
Expected shape:  torch.Size([1, 3, 5, 5])
Expected shape:  (5, 5, 3)
test_output shape:  (5, 5, 3)


AssertionError: 

In [32]:
def PaddedDepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    # out = np.zeros((outRows, outCols, nFilters))
    Input = [[[str(int(input[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[str(int(weights[i][j][k].round()) % p) for k in range(weights.shape[2])] for j in range(weights.shape[1])] for i in range(weights.shape[0])]
    out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    Bias = bias.round().astype(int).astype(str).tolist()
    out_str = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    remainder = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row+1][col+1][channel] += int(input[row*strides+x][col*strides+y][channel]) * int(weights[x, y, channel])
                
                out[row+1][col+1][channel] += int(bias[channel])
                remainder[row+1][col+1][channel] = str(int(out[row+1][col+1][channel] % n))
                out[row+1][col+1][channel] = int(out[row+1][col+1][channel] // n)
                out_str[row+1][col+1][channel] = str(out[row+1][col+1][channel] % p)
                            
    return Input, Weights, Bias, out_str, out, remainder
    
# def MobilenetDepthwiseConv(nRows, nCols, nChannels, nFilters, n, input, weights, bias):
#     kernelSize = 3
#     strides = 1
    
#     assert(nFilters % nChannels == 0)
#     outRows = (nRows - kernelSize)//strides + 1
#     outCols = (nCols - kernelSize)//strides + 1
    
#     # out = np.zeros((outRows, outCols, nFilters))
#     Input = [[[str(int(input[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
#     Weights = [[[str(int(weights[i][j][k].round()) % p) for k in range(weights.shape[2])] for j in range(weights.shape[1])] for i in range(weights.shape[0])]
#     out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
#     Bias = bias.round().astype(int).astype(str).tolist()
#     out_str = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
#     remainder = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    
#     for row in range(outRows):
#         for col in range(outCols):
#             for channel in range(nChannels):
#                 for x in range(kernelSize):
#                     for y in range(kernelSize):
#                         out[row+1][col+1][channel] += int(input[row*strides+x][col*strides+y][channel]) * int(weights[x, y, channel])
                
#                 out[row+1][col+1][channel] += int(bias[channel])
#                 remainder[row+1][col+1][channel] = str(int(out[row+1][col+1][channel] % n))
#                 out[row+1][col+1][channel] = int(out[row+1][col+1][channel] // n)
#                 out_str[row+1][col+1][channel] = str(out[row+1][col+1][channel] % p)
                            
#     return Input, Weights, Bias, out_str, out, remainder

In [37]:
def PointwiseConv2d(nRows, nCols, nChannels, nFilters, strides, n, input, weights, bias):
    kernelSize = 1
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    # out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    # remainder = [[[None for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    
    Input = [[[str(int(input[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    # Weights = [[[str(int(weights[i][j][k].round()) % p) for k in range(weights.shape[2])] for j in range(weights.shape[1])] for i in range(weights.shape[0])]
    Weights = [[str(int(weights[i][j].round()) % p)for j in range(weights.shape[1])] for i in range(weights.shape[0])]
    out = [[[0 for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    Bias = bias.round().astype(int).astype(str).tolist()
    out_str = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    remainder = [[[str(0) for _ in range(nFilters)] for _ in range(nCols)] for _ in range(nRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for filter in range(nFilters):
                for k in range(nChannels):
                    out[row][col][filter] += int(input[row*strides][col*strides][k]) * int(weights[k, filter])
                    
                out[row][col][filter] += int(bias[filter])
                remainder[row][col][filter] = str(int(out[row][col][filter] % n))
                out[row][col][filter] = int(out[row][col][filter] // n)
                out_str[row][col][filter] = str(out[row][col][filter] % p)
                            
    # return out, remainder
    return Input, Weights, Bias, out_str, out, remainder

In [38]:
# depth_weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
# depth_bias = torch.zeros(depth_weights.shape[0]).numpy()

# print("Input shape: ", input.shape)
# expected = test_model.dw_conv[0](input).detach()
# print("Expected shape: ", expected.shape)

# depth_weights = depth_weights.transpose((1, 2, 0))

# quantized_image = input.squeeze().numpy().transpose((1,2,0)) * 10**EXPONENT
# # quantized_image = padded * 10**EXPONENT
# quantized_depth_weights = depth_weights * 10**EXPONENT

# circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)

# test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in depth_out])
# test_output = test_output[1:6, 1:6, :]

# expected = expected.squeeze().numpy().transpose((1,2,0))

# assert(np.allclose(expected, test_output, atol=0.00001))

# # point_weights = model.pw_conv[0].weight.squeeze().detach().numpy()
# point_weights = test_model.pw_conv[0].weight.detach().numpy()
# print("point weights shape: ", point_weights.shape)
# point_bias = torch.zeros(point_weights.shape[0]).numpy()

# print("Input shape: ", input.shape)
# depth_expected = test_model.dw_conv[0](input)
# point_expected = test_model.pw_conv[0](depth_expected)
# print("Depth Expected shape: ", depth_expected.shape)
# # print("Point Expected shape: ", point_expected.shape)

# point_expected = point_expected.squeeze().detach().numpy().transpose((1,2,0))
# print("Point Expected shape: ", point_expected.shape)

# point_weights = point_weights.transpose((2, 3, 1, 0)).squeeze()
# quantized_point_weights = point_weights * 10**EXPONENT
# print("point weights shape: ", quantized_point_weights.shape)

# # circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)
# point_input, circuit_point_weights, circuit_point_bias, circuit_point_out, point_out, circuit_point_remainder = PointwiseConv2d(7, 7, 3, 6, 1, 10**EXPONENT, depth_out, quantized_point_weights.round(), point_bias)

# test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in point_out])
# test_output = test_output[1:6, 1:6, :]

# assert(np.allclose(point_expected, test_output, atol=0.00001))

# input_json_path = "padded_input.json"
# with open(input_json_path, "w") as input_file:
#     json.dump({
#                "in": circuit_in,
#                "dw_conv_weights": circuit_depth_weights,
#                "dw_conv_bias": circuit_depth_bias,
#                "dw_conv_remainder": circuit_depth_remainder,
#                "dw_conv_out": circuit_depth_out,
        
#                "pw_conv_weights": circuit_point_weights,
#                "pw_conv_bias": circuit_point_bias,
#                "pw_conv_remainder": circuit_point_remainder,
#                "pw_conv_out": circuit_point_out,
#               },
#               input_file)


# os.chdir("circuits")
# !./padded/padded_cpp/padded ../padded_input.json head.wtns
# # # !npx snarkjs groth16 prove ./origDepthwiseConv2d/circuit_final.zkey head.wtns proof.json public_test.json
# os.chdir("../")

# print("OK")

Input shape:  torch.Size([1, 3, 7, 7])
Expected shape:  torch.Size([1, 3, 5, 5])
point weights shape:  (6, 3, 1, 1)
Input shape:  torch.Size([1, 3, 7, 7])
Depth Expected shape:  torch.Size([1, 3, 5, 5])
Point Expected shape:  (5, 5, 6)
point weights shape:  (3, 6)
Not all inputs have been set. Only 1083 out of 1983
padded: main.cpp:268: int main(int, char**): Assertion `false' failed.
OK


In [603]:
model.dw_conv[0].weight.squeeze().detach().numpy().shape, model.pw_conv[0].weight.squeeze().detach().numpy().shape

((3, 3, 3), (6, 3))

In [43]:
# Depthwise convolution
depth_weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
depth_bias = torch.zeros(depth_weights.shape[0]).numpy()

print("Input shape: ", input.shape)
expected = test_model.dw_conv[0](input).detach()
print("Expected shape: ", expected.shape)

depth_weights = depth_weights.transpose((1, 2, 0))

quantized_image = input.squeeze().numpy().transpose((1,2,0)) * 10**EXPONENT
# quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depth_weights * 10**EXPONENT

circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in depth_out])
test_output = test_output[1:6, 1:6, :]

expected = expected.squeeze().numpy().transpose((1,2,0))

assert(np.allclose(expected, test_output, atol=0.00001))

# Batch normalization step
# BATCH NORM CONSTANTS
test_model.eval()
gamma = test_model.dw_conv[1].weight
beta = test_model.dw_conv[1].bias
mean = test_model.dw_conv[1].running_mean
var = test_model.dw_conv[1].running_var
eps = test_model.dw_conv[1].eps

a = (gamma/(var+eps)**.5).detach()
print('a shape: ', a.shape)
b = (beta-gamma*mean/(var+eps)**.5).detach()
print('b shape: ', b.shape)

b = b.tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

out = test_model.dw_conv[0](input)
expected = test_model.dw_conv[1](out)

expected = torch.permute(expected.squeeze(), (1, 2, 0))

quantized_in = torch.permute(out.squeeze(), (1, 2, 0)) * 10**EXPONENT

X, A, B, actual, remainder = BatchNormalizationInt(5, 5, 3, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# quantized_in = [[[from_circom(int(out)) for out in vec] for vec in matrix] for matrix in circuit_conv_out]

_, circuit_depth_bn_a, circuit_depth_bn_b, circuit_depth_bn_out, circuit_depth_bn_remainder = BatchNormalizationInt(7, 7, 3, 10**EXPONENT, depth_out, quantized_a, quantized_b)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_depth_bn_out])
test_output = test_output[1:6, 1:6, :]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

Input shape:  torch.Size([1, 3, 7, 7])
Expected shape:  torch.Size([1, 3, 5, 5])
a shape:  torch.Size([3])
b shape:  torch.Size([3])


In [60]:
# Depthwise convolution
input = torch.randn((1, 3, 7, 7))

depth_weights = test_model.dw_conv[0].weight.squeeze().detach().numpy()
depth_bias = torch.zeros(depth_weights.shape[0]).numpy()

print("Input shape: ", input.shape)
expected = test_model.dw_conv[0](input).detach()
print("Expected shape: ", expected.shape)

depth_weights = depth_weights.transpose((1, 2, 0))

quantized_image = input.squeeze().numpy().transpose((1,2,0)) * 10**EXPONENT
# quantized_image = padded * 10**EXPONENT
quantized_depth_weights = depth_weights * 10**EXPONENT

circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in depth_out])
test_output = test_output[1:-1, 1:-1, :]

expected = expected.squeeze().numpy().transpose((1,2,0))

assert(np.allclose(expected, test_output, atol=0.00001))

# Batch normalization step
# BATCH NORM CONSTANTS
test_model.eval()
gamma = test_model.dw_conv[1].weight
beta = test_model.dw_conv[1].bias
mean = test_model.dw_conv[1].running_mean
var = test_model.dw_conv[1].running_var
eps = test_model.dw_conv[1].eps

a = (gamma/(var+eps)**.5).detach()
print('a shape: ', a.shape)
b = (beta-gamma*mean/(var+eps)**.5).detach()
print('b shape: ', b.shape)

b = b.tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

out = test_model.dw_conv[0](input)
expected = test_model.dw_conv[1](out)

expected = torch.permute(expected.squeeze(), (1, 2, 0))

quantized_in = torch.permute(out.squeeze(), (1, 2, 0)) * 10**EXPONENT

X, A, B, actual, remainder = BatchNormalizationInt(5, 5, 3, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# quantized_in = [[[from_circom(int(out)) for out in vec] for vec in matrix] for matrix in circuit_conv_out]

_, circuit_depth_bn_a, circuit_depth_bn_b, circuit_depth_bn_out, circuit_depth_bn_remainder = BatchNormalizationInt(7, 7, 3, 10**EXPONENT, depth_out, quantized_a, quantized_b)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_depth_bn_out])
test_output = test_output[1:-1, 1:-1, :]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# Pointwise convolution
# point_weights = test_model.pw_conv[0].weight.squeeze().detach().numpy()
point_weights = test_model.pw_conv[0].weight.detach().numpy()
print("point weights shape: ", point_weights.shape)
point_bias = torch.zeros(point_weights.shape[0]).numpy()

print("Input shape: ", input.shape)
depth_expected = test_model.dw_conv[0](input)
bn_expected = test_model.dw_conv[1](depth_expected)
point_expected = test_model.pw_conv[0](bn_expected)
print("Depth Expected shape: ", depth_expected.shape)
# print("Point Expected shape: ", point_expected.shape)

point_expected = point_expected.squeeze().detach().numpy().transpose((1,2,0))
print("Point Expected shape: ", point_expected.shape)

point_weights = point_weights.transpose((2, 3, 1, 0)).squeeze()
quantized_point_weights = point_weights * 10**EXPONENT
print("point weights shape: ", quantized_point_weights.shape)

# circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)
point_input, circuit_point_weights, circuit_point_bias, circuit_point_out, point_out, circuit_point_remainder = PointwiseConv2d(7, 7, 3, 6, 1, 10**EXPONENT, circuit_depth_bn_out, quantized_point_weights.round(), point_bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in point_out])
test_output = test_output[1:-1, 1:-1, :]

assert(np.allclose(point_expected, test_output, atol=1e-6))

                   
# Batch normalization step
# BATCH NORM CONSTANTS
test_model.eval()
gamma = test_model.pw_conv[1].weight
beta = test_model.pw_conv[1].bias
mean = test_model.pw_conv[1].running_mean
var = test_model.pw_conv[1].running_var
eps = test_model.pw_conv[1].eps

a = (gamma/(var+eps)**.5).detach()
print('a shape: ', a.shape)
b = (beta-gamma*mean/(var+eps)**.5).detach()
print('b shape: ', b.shape)

b = b.tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

# out = test_model.dw_conv[0](input)
# expected = test_model.dw_conv[1](out)
        
print("Input shape: ", input.shape)
depth_expected = test_model.dw_conv[0](input)
bn_expected = test_model.dw_conv[1](depth_expected)
point_expected = test_model.pw_conv[0](bn_expected)
expected = test_model.pw_conv[1](point_expected)
print("Depth Expected shape: ", expected.shape)

              
expected = torch.permute(expected.squeeze(), (1, 2, 0))

quantized_in = torch.permute(point_expected.squeeze(), (1, 2, 0)) * 10**EXPONENT

X, A, B, actual, remainder = BatchNormalizationInt(5, 5, 6, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

_, circuit_point_bn_a, circuit_point_bn_b, circuit_point_bn_out, circuit_point_bn_remainder = BatchNormalizationInt(7, 7, 6, 10**EXPONENT, point_out, quantized_a, quantized_b)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_point_bn_out])
test_output = test_output[1:-1, 1:-1, :]

print("Expected shape: ", expected.shape)
print("test shape: ", test_output.shape)
assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

input_json_path = "padded_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({
               "in": circuit_in,
               "dw_conv_weights": circuit_depth_weights,
               "dw_conv_bias": circuit_depth_bias,
               "dw_conv_remainder": circuit_depth_remainder,
               "dw_conv_out": circuit_depth_out,
        
               "dw_bn_a": circuit_depth_bn_a,
               "dw_bn_b": circuit_depth_bn_b,
               "dw_bn_remainder": circuit_depth_bn_remainder,
               "dw_bn_out": circuit_depth_bn_out,
        
               "pw_conv_weights": circuit_point_weights,
               "pw_conv_bias": circuit_point_bias,
               "pw_conv_remainder": circuit_point_remainder,
               "pw_conv_out": circuit_point_out,
        
               "pw_bn_a": circuit_point_bn_a,
               "pw_bn_b": circuit_point_bn_b,
               "pw_bn_remainder": circuit_point_bn_remainder,
               "pw_bn_out": circuit_point_bn_out,
              },
              input_file)


os.chdir("circuits")
!./padded/padded_cpp/padded ../padded_input.json head.wtns
# # !npx snarkjs groth16 prove ./origDepthwiseConv2d/circuit_final.zkey head.wtns proof.json public_test.json
os.chdir("../")

print("OK")

Input shape:  torch.Size([1, 3, 7, 7])
Expected shape:  torch.Size([1, 3, 5, 5])
a shape:  torch.Size([3])
b shape:  torch.Size([3])
point weights shape:  (6, 3, 1, 1)
Input shape:  torch.Size([1, 3, 7, 7])
Depth Expected shape:  torch.Size([1, 3, 5, 5])
Point Expected shape:  (5, 5, 6)
point weights shape:  (3, 6)
a shape:  torch.Size([6])
b shape:  torch.Size([6])
Input shape:  torch.Size([1, 3, 7, 7])
Depth Expected shape:  torch.Size([1, 6, 5, 5])
Expected shape:  torch.Size([5, 5, 6])
test shape:  (5, 5, 6)
START
PRIME - 1 28948022309329048855892746252171976963363056481941647379679742748393362948096
------------------------------------------------
------------------------------------------------
dw_conv done
depth batch norm done
pw_conv done
point batch norm done
END
OK


# Padded convolution 2 iterations test with true input

In [47]:
# MODEL_WEIGHTS_PATH = './checkpoints/model_small_100epochs.pth'

# model = ZkMobileNet(trainloader, num_classes=10, alpha=0.25, max_epochs=100)
# checkpoint = torch.load(MODEL_WEIGHTS_PATH)
# # model.load_state_dict(checkpoint['state_dict'])
# model.load_state_dict(checkpoint['net'])
model.eval()

image, label = validset[0]
image = image.unsqueeze(0)
logits = model(image)
pred_idx = logits.argmax()

print(f"Predicted {classes[pred_idx]} - idx: {pred_idx}")

Predicted horse - idx: 7


In [163]:
# CONVOLUTION LAYER
weights = model.conv.weight.detach().numpy().transpose(2, 3, 1, 0)
image, label = testset[0]

expected = model.conv(image).detach().numpy()
bias = torch.zeros(weights.shape[3]).numpy()

# padded = pad(image, 1).transpose(1,2,0)
padded = F.pad(image, (1,1,1,1), "constant", 0).numpy()
padded = padded.transpose(1,2,0)

quantized_image = (padded * 10**EXPONENT).round()
quantized_weights = (weights * 10**EXPONENT).round() # .transpose(0, 3, 1, 0) # [nFilters, nChannels, H, W] -> 

circuit_in, circuit_conv_weights, circuit_conv_bias, circuit_conv_out, circuit_conv_remainder = Conv2DInt(34, 34, 3, 8, 3, 1, 10**EXPONENT, quantized_image, quantized_weights, bias)
# output, remainder = Conv2DInt(34, 34, 3, 8, 3, 1, 10**EXPONENT, quantized_image, quantized_weights, bias)
# test_output = output / 10**(EXPONENT)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_conv_out]

expected = expected.transpose((1, 2, 0))

assert(np.allclose(test_output, expected, atol=1e-6))

# BATCH NORM CONSTANTS
gamma = model.bn.weight
beta = model.bn.bias
mean = model.bn.running_mean
var = model.bn.running_var
eps = model.bn.eps

a = (gamma/(var+eps)**.5).detach()
b = (beta-gamma*mean/(var+eps)**.5).detach().tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

# BATCH NORM USING PYTORCH OUTPUT
image, label = testset[0]
image = image.unsqueeze(0)
out = model.conv(image)
expected = model.bn(out)

expected = torch.permute(expected.squeeze(), (1, 2, 0))

quantized_in = torch.permute(out.squeeze(), (1, 2, 0)) * 10**EXPONENT

X, A, B, actual, remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, quantized_in, quantized_a, quantized_b)
test_output = [[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in actual]
assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# BATCH NORM USING CIRCUIT CONV OUTPUT
quantized_in = [[[from_circom(int(out)) for out in vec] for vec in matrix] for matrix in circuit_conv_out]

_, circuit_bn_a, circuit_bn_b, circuit_bn_out, circuit_bn_remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, quantized_in, quantized_a, quantized_b)

test_output = [[[int(from_circom(int(out))) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in circuit_bn_out]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-6))

# RELU USING CIRCUIT OUTPUT
relu_in = [[[to_circom(bn_out) if bn_out < MAX_POSITIVE else 0 for bn_out in vec] for vec in matrix] for matrix in circuit_bn_out]
relu_out = [[[str(bn_out) if bn_out < MAX_POSITIVE else 0 for bn_out in vec] for vec in matrix] for matrix in relu_in]

image, label = testset[0]
image = image.unsqueeze(0)
conv_output = model.conv(image)
bn_output = model.bn(conv_output)
relu_expected = model.relu(bn_output).squeeze().detach().numpy().transpose((1,2,0))
print(relu_expected.shape)
print(dw_circuit_input.shape)

test_output = [[[out / 10**EXPONENT if out > 0 else 0 for out in vec] for vec in matrix] for matrix in circuit_bn_out]

assert(np.allclose(test_output, relu_expected, atol=1e-6))

(32, 32, 8)
(32, 32, 8)


# DW INPUT WITH ACTUAL IMAGE

# Depthwise convolution step

In [321]:
# Depthwise convolution
input = torch.Tensor([[[out if out > 0 else 0 for out in vec] for vec in matrix] for matrix in circuit_bn_out])
pytorch_input = torch.Tensor([[[int(out) / 10**EXPONENT if out > 0 else 0 for out in vec] for vec in matrix] for matrix in circuit_bn_out])
pytorch_input = torch.permute(pytorch_input, (2, 0, 1)).unsqueeze(0)
print("Input shape: ", input.shape)

depth_weights = model.features[0].dw_conv[0].weight.squeeze().detach().numpy()
depth_bias = torch.zeros(depth_weights.shape[0]).numpy()

print("Pytorch Input shape: ", pytorch_input.shape)
dw_conv_pytorch_output = model.features[0].dw_conv[0](pytorch_input).detach()
print("Expected shape: ", expected.shape)

depth_weights = depth_weights.transpose((1, 2, 0))
quantized_depth_weights = depth_weights * 10**EXPONENT

circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(32, 32, 8, 8, 3, 1, 10**EXPONENT, input, quantized_depth_weights.round(), depth_bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in depth_out])
test_output = test_output[1:31, 1:31, :]

expected = dw_conv_pytorch_output.squeeze().numpy().transpose((1,2,0))

assert(np.allclose(expected, test_output, atol=0.00001))


Input shape:  torch.Size([32, 32, 8])
Pytorch Input shape:  torch.Size([1, 8, 32, 32])
Expected shape:  torch.Size([30, 30, 16])


# Depthwise batch normalization step

In [339]:
# Batch normalization step
# BATCH NORM CONSTANTS
model.eval()
gamma = model.features[0].dw_conv[1].weight
beta = model.features[0].dw_conv[1].bias
mean = model.features[0].dw_conv[1].running_mean
var = model.features[0].dw_conv[1].running_var
eps = model.features[0].dw_conv[1].eps

a = (gamma/(var+eps)**.5).detach()
print('a shape: ', a.shape)
b = (beta-gamma*mean/(var+eps)**.5).detach()
print('b shape: ', b.shape)

b = b.tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

# out = model.features[0].dw_conv[0](input)
dw_conv_pytorch_output = model.features[0].dw_conv[0](pytorch_input).detach()
dw_bn_pytorch_output = model.features[0].dw_conv[1](dw_conv_pytorch_output)

expected = torch.permute(dw_bn_pytorch_output.squeeze(), (1, 2, 0))

_, circuit_depth_bn_a, circuit_depth_bn_b, circuit_depth_bn_out, circuit_depth_bn_remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, depth_out, quantized_a, quantized_b)

test_output = np.array([[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in circuit_depth_bn_out])
test_output = test_output[1:-1, 1:-1, :]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-5))

a shape:  torch.Size([8])
b shape:  torch.Size([8])


# Pointwise convolution step

In [340]:
# point_weights = test_model.pw_conv[0].weight.squeeze().detach().numpy()
point_weights = model.features[0].pw_conv[0].weight.detach().numpy()
print("point weights shape: ", point_weights.shape)
point_bias = torch.zeros(point_weights.shape[0]).numpy()

print("Input shape: ", input.shape)
# depth_expected = model.dw_conv[0](input)
# bn_expected = test_model.dw_conv[1](depth_expected)
# point_expected = test_model.pw_conv[0](bn_expected)

dw_conv_pytorch_output = model.features[0].dw_conv[0](pytorch_input).detach()
dw_bn_pytorch_output = model.features[0].dw_conv[1](dw_conv_pytorch_output)
pw_conv_pytorch_output = model.features[0].pw_conv[0](dw_bn_pytorch_output)

expected = torch.permute(pw_conv_pytorch_output.squeeze(), (1, 2, 0))

print("point Expected shape: ", expected.shape)
# print("Point Expected shape: ", point_expected.shape)

# point_expected = point_expected.squeeze().detach().numpy().transpose((1,2,0))
# print("Point Expected shape: ", point_expected.shape)

point_weights = point_weights.transpose((2, 3, 1, 0)).squeeze()
quantized_point_weights = point_weights * 10**EXPONENT
print("point weights shape: ", quantized_point_weights.shape)

# circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)
point_input, circuit_point_weights, circuit_point_bias, circuit_point_out, point_out, circuit_point_remainder = PointwiseConv2d(32, 32, 8, 16, 1, 10**EXPONENT, circuit_depth_bn_out, quantized_point_weights.round(), point_bias)

test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in point_out])
test_output = test_output[1:-1, 1:-1, :]

assert(np.allclose(expected.detach().numpy(), test_output, atol=1e-4))

point weights shape:  (16, 8, 1, 1)
Input shape:  torch.Size([32, 32, 8])
point Expected shape:  torch.Size([30, 30, 16])
point weights shape:  (8, 16)


# Pointwise batch normalization step

In [338]:
# Batch normalization step
# BATCH NORM CONSTANTS
model.eval()
gamma = model.features[0].pw_conv[1].weight
beta = model.features[0].pw_conv[1].bias
mean = model.features[0].pw_conv[1].running_mean
var = model.features[0].pw_conv[1].running_var
eps = model.features[0].pw_conv[1].eps

a = (gamma/(var+eps)**.5).detach()
print('a shape: ', a.shape)
b = (beta-gamma*mean/(var+eps)**.5).detach()
print('b shape: ', b.shape)

b = b.tolist()

quantized_a = (a * 10**EXPONENT).tolist()
quantized_b = [bi * 10**(2*EXPONENT) for bi in b]

# out = model.features[0].dw_conv[0](input)
dw_conv_pytorch_output = model.features[0].dw_conv[0](pytorch_input).detach()
dw_bn_pytorch_output = model.features[0].dw_conv[1](dw_conv_pytorch_output)

pw_conv_pytorch_output = model.features[0].pw_conv[0](dw_bn_pytorch_output)
pw_bn_pytorch_output = model.features[0].pw_conv[1](pw_conv_pytorch_output)

expected = torch.permute(pw_bn_pytorch_output.squeeze(), (1, 2, 0))

_, circuit_point_bn_a, circuit_point_bn_b, circuit_point_bn_out, circuit_point_bn_remainder = BatchNormalizationInt(32, 32, 16, 10**EXPONENT, point_out, quantized_a, quantized_b)

test_output = np.array([[[from_circom(int(out)) / 10**EXPONENT for out in vec] for vec in matrix] for matrix in circuit_point_bn_out])
print("TEST OUTPUT SHAPE: ", test_output.shape)
print("EXPECTED SHAPE: ", expected.shape)
test_output = test_output[1:-1, 1:-1, :]

assert(torch.allclose(torch.Tensor(test_output), expected, atol=1e-5))

a shape:  torch.Size([16])
b shape:  torch.Size([16])
TEST OUTPUT SHAPE:  (32, 32, 16)
EXPECTED SHAPE:  torch.Size([30, 30, 16])


In [336]:
len(circuit_depth_bn_a)

16

In [346]:
input_json_path = "test.json"
with open(input_json_path, "w") as input_file:
    json.dump({
               "in": circuit_in,
               "dw_conv_weights": circuit_depth_weights,
               "dw_conv_bias": circuit_depth_bias,
               "dw_conv_remainder": circuit_depth_remainder,
               "dw_conv_out": circuit_depth_out,
        
               "dw_bn_a": circuit_depth_bn_a,
               "dw_bn_b": circuit_depth_bn_b,
               "dw_bn_remainder": circuit_depth_bn_remainder,
               "dw_bn_out": circuit_depth_bn_out,
        
               "pw_conv_weights": circuit_point_weights,
               "pw_conv_bias": circuit_point_bias,
               "pw_conv_remainder": circuit_point_remainder,
               "pw_conv_out": circuit_point_out,
        
               "pw_bn_a": circuit_point_bn_a,
               "pw_bn_b": circuit_point_bn_b,
               "pw_bn_remainder": circuit_point_bn_remainder,
               "pw_bn_out": circuit_point_bn_out,
              },
              input_file)


os.chdir("circuits")
!./padded/padded_cpp/padded ../test.json head.wtns
# # !npx snarkjs groth16 prove ./origDepthwiseConv2d/circuit_final.zkey head.wtns proof.json public_test.json
os.chdir("../")

print("OK")

START
PRIME - 1 28948022309329048855892746252171976963363056481941647379679742748393362948096
------------------------------------------------
------------------------------------------------
dw_conv done
depth batch norm done
pw_conv done
point batch norm done
END
OK


In [135]:
input_json_path = "head_input1.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": circuit_in,
               "conv2d_weights": circuit_conv_weights,
               "conv2d_bias": circuit_conv_bias,
               "conv2d_out": circuit_conv_out,
               "conv2d_remainder": circuit_conv_remainder,
               
               "bn_a": circuit_bn_a,
               "bn_b": circuit_bn_b,
               "bn_out": circuit_bn_out,
               "bn_remainder": circuit_bn_remainder,
               
               "relu_out": relu_out,
               }, input_file)

os.chdir("circuits")
!./head/head_cpp/head ../head_input1.json head.wtns
# !npx snarkjs groth16 prove head/circuit_final.zkey head.wtns proof.json public_test.json
os.chdir("../")

print("ok")

something
before conv2d
after conv2d
after bn
at:  0 1 6
in:  1745789835369983
after relu
end
ok


In [88]:
image, label = testset[0]
image = image.unsqueeze(0)
conv_out = model.conv(image)
bn_out = model.bn(conv_out)
dw_input = model.relu(bn_out)
dw_conv_expected_output = model.features[0].dw_conv[0](dw_input)
dw_bn_expected_output = model.features[0].dw_conv[1](dw_conv_expected_output)

dw_conv_expected_output = dw_conv_expected_output.detach().numpy().squeeze().transpose((1,2,0))
dw_bn_expected_output = dw_bn_expected_output.detach().numpy().squeeze().transpose((1,2,0))
print(dw_input.shape)
print(expected_output.shape)

torch.Size([1, 8, 32, 32])
(30, 30, 8)


In [226]:
# def BatchNormalizationInt(nRows, nCols, nChannels, n, X_in, a_in, b_in):
#     p = CIRCOM_PRIME
#     X = [[[str(int(X_in[i][j][k]) % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
#     A = [str(int(a_in[k]) % p) for k in range(nChannels)]
#     B = [str(int(b_in[k]) % p) for k in range(nChannels)]
#     out = [[[0 for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
#     remainder = [[[None for _ in range(nChannels)] for _ in range(nCols)] for _ in range(nRows)]
#     for i in range(nRows):
#         for j in range(nCols):
#             for k in range(nChannels):
#                 out[i][j][k] = int(int(X_in[i][j][k]) * int(a_in[k]) + int(b_in[k]))
#                 remainder[i][j][k] = str(int(out[i][j][k]) % n)
#                 out[i][j][k] = int(out[i][j][k] // n)
#     return X, A, B, out, remainder
    
class CleanUp():
    def __init__(self, model: ZkMobileNet, image: torch.Tensor):
        ''' Image should be 3d tensor to start '''
        # self.image = image.unsqueeze(0)
        self.model = model
        self.model.eval()
        # conv_out = model.conv(self.image)
        # bn_out = model.bn(conv_out)
        # self.relu_out = model.relu(bn_out)
        self.dw_conv_expected = model.features[0].dw_conv[0](image)
        self.dw_bn_expected = model.features[0].dw_conv[1](self.dw_conv_expected)
        
        self.pw_conv_expected = model.features[0].pw_conv[0](self.dw_bn_expected)
        self.pw_bn_expected = model.features[0].pw_conv[1](self.pw_conv_expected)
        
        self.dw_conv_expected = self.dw_conv_expected.squeeze().detach().numpy().transpose((1,2,0))
        self.dw_bn_expected = self.dw_bn_expected.squeeze().detach().numpy().transpose((1,2,0))
        
        self.pw_conv_expected = self.pw_conv_expected.squeeze().detach().numpy().transpose((1,2,0))
        self.pw_bn_expected = self.pw_bn_expected.squeeze().detach().numpy().transpose((1,2,0))
        
        # return self;

    def get_depthwise_inputs(self, input):
        conv_inputs = helper.get_depthwise_conv_inputs(input)
        bn_inputs = helper.get_depthwise_bn_inputs(conv_inputs['out'])
        return { 
            "conv": conv_inputs, 
            "bn": bn_inputs 
        } 
        
    def get_depthwise_conv_inputs(self, input):
        depth_weights = self.model.features[0].dw_conv[0].weight.squeeze().detach().numpy()
        depth_bias = torch.zeros(depth_weights.shape[0]).numpy()
        
        depth_weights = depth_weights.transpose((1, 2, 0))
        quantized_depth_weights = depth_weights * 10**EXPONENT
        
        circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(32, 32, 8, 8, 3, 1, 10**EXPONENT, input, quantized_depth_weights.round(), depth_bias)
        
        test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in depth_out])
        test_output = test_output[1:-1, 1:-1, :]
        
        assert(np.allclose(self.dw_conv_expected, test_output, atol=0.00001))
        return { 
            "in": circuit_in,
            "weights": circuit_depth_weights,
            "bias": circuit_depth_bias,
            "out_str": circuit_depth_out,
            "out": depth_out,
            "remainder": circuit_depth_remainder,
        }

    def get_depthwise_bn_inputs(self, input):
        gamma = self.model.features[0].dw_conv[1].weight
        beta = self.model.features[0].dw_conv[1].bias
        mean = self.model.features[0].dw_conv[1].running_mean
        var = self.model.features[0].dw_conv[1].running_var
        eps = self.model.features[0].dw_conv[1].eps
        
        a = (gamma/(var+eps)**.5).detach()
        b = (beta-gamma*mean/(var+eps)**.5).detach()
        
        b = b.tolist()
        
        quantized_a = (a * 10**EXPONENT).tolist()
        quantized_b = [bi * 10**(2*EXPONENT) for bi in b]
        
        _, circuit_depth_bn_a, circuit_depth_bn_b, circuit_depth_bn_out, circuit_depth_bn_remainder = BatchNormalizationInt(32, 32, 8, 10**EXPONENT, input, quantized_a, quantized_b)
        
        test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in circuit_point_bn_out])
        test_output = test_output[1:-1, 1:-1, :]
        
        # print("Expected shape: ", dw_bn_expected_output.shape)
        # print("test shape: ", test_output.shape)
        assert(np.allclose(test_output, self.dw_bn_expected, atol=1e-5))
        
        return { 
            "a": circuit_depth_bn_a,
            "b": circuit_depth_bn_b,
            "out": circuit_depth_out,
            "remainder": circuit_depth_remainder,
        }

    def get_pointwise_conv_inputs(self, input):
        # point_weights = model.pw_conv[0].weight.detach().numpy()
        point_weights = self.model.features[0].pw_conv[0].weight.detach().numpy()
        print("point weights shape: ", point_weights.shape)
        point_bias = torch.zeros(point_weights.shape[0]).numpy()
        
        # print("Input shape: ", input.shape)
        # depth_expected = test_model.dw_conv[0](input)
        # bn_expected = test_model.dw_conv[1](depth_expected)
        # point_expected = test_model.pw_conv[0](bn_expected)
        # print("Depth Expected shape: ", depth_expected.shape)
        # print("Point Expected shape: ", point_expected.shape)
        
        # point_expected = point_expected.squeeze().detach().numpy().transpose((1,2,0))
        # print("Point Expected shape: ", point_expected.shape)
        
        point_weights = point_weights.transpose((2, 3, 1, 0)).squeeze()
        quantized_point_weights = point_weights * 10**EXPONENT
        print("point weights shape: ", quantized_point_weights.shape)
        
        # circuit_in, circuit_depth_weights, circuit_depth_bias, circuit_depth_out, depth_out, circuit_depth_remainder = PaddedDepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_depth_weights.round(), depth_bias)
        # point_input, circuit_point_weights, circuit_point_bias, circuit_point_out, point_out, circuit_point_remainder = PointwiseConv2d(7, 7, 3, 6, 1, 10**EXPONENT, circuit_depth_bn_out, quantized_point_weights.round(), point_bias)
        point_input, circuit_point_weights, circuit_point_bias, circuit_point_out, point_out, circuit_point_remainder = PointwiseConv2d(32, 32, 8, 16, 1, 10**EXPONENT, input, quantized_point_weights.round(), point_bias)
        
        test_output = np.array([[[int(out) / 10**EXPONENT for out in asdf] for asdf in asdfasdf] for asdfasdf in point_out])
        test_output = test_output[1:-1, 1:-1, :]
        print("EXPECTED SHAPE: ", self.pw_conv_expected.shape)
        print("ACTUAL SHAPE: ", test_output.shape)
        print(self.pw_conv_expected[0][0])
        print("---------------------------------------------")
        print(test_output[1][1])
        
        assert(np.allclose(self.pw_conv_expected, test_output, atol=1e-6))

image, _ = testset[0]
conv_out = model.conv(image.unsqueeze(0))
bn_out = model.bn(conv_out)
relu_out = model.relu(bn_out)
helper = CleanUp(model, relu_out)

dw_conv_input = [[[out if out > 0 else 0 for out in vec] for vec in matrix] for matrix in circuit_bn_out]
circuit_depth_inputs = helper.get_depthwise_inputs(dw_conv_input)
print(helper.pw_conv_expected.shape)
print("shape: ", np.array(circuit_depth_inputs['bn']['out']).shape)
# circuit_point_conv_inputs = helper.get_pointwise_conv_inputs(circuit_depth_inputs['bn']['out'])

(30, 30, 16)
shape:  (32, 32, 8)
